In [4]:
!python -m pip install paddlepaddle-gpu==2.1.3.post101 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install "paddleocr>=2.0.1"

import pandas as pd
import re
import numpy as np
import cv2
from PIL import Image
import math
import os

from paddleocr import PaddleOCR, draw_ocr


/usr/local/lib/python3.7/dist-packages/skimage/data/__init__.py:107: DeprecationWarning: 
    Importing file_hash from pooch.utils is DEPRECATED. Please import from the
    top-level namespace (`from pooch import file_hash`) instead, which is fully
    backwards compatible with pooch >= 0.1.
    
  return file_hash(path) == expected_hash


## 偵測: YOLOv4
    需先設定好YOLOv4環境

    需準備文件: obj.data / yolov4_ad_test.cfg
    圖檔位置txt: test_real_private.txt
    訓練完模型: yolov4_ad_best.weights

    產出: result_real_private.txt

        -- 可不執行; 已直接附上當天結果result_real_private.txt

In [6]:
# 製作test_real_private.txt 供YOLOv4預測用
df_test_ = pd.read_csv(r'/content/drive/MyDrive/t_brain/submission_template_private.csv')
path = f"test_real_private.txt"
with open(path, 'w') as f:
    for _,x in df_test_.iterrows():
        f.write(f"/content/drive/MyDrive/t_brain/private/data/private_data_v2/{x['id']}.jpg\n")

In [ ]:
# 批次檢驗; 會輸出result_real_private.txt
!chmod +x /content/drive/MyDrive/yolo_file/darknet-master/darknet
!cd /content/drive/MyDrive/yolo_file/darknet-master; ./darknet detector test cfg_steel/obj.data cfg_steel/yolov4_ad_test.cfg cfg_steel/weights/yolov4_ad_best.weights -ext_output -dont_show <cfg_steel/test_real_private.txt> result_real_private.txt -gpus 0,1,2,3

## 識別: PaddleOCR

### 取YOLO結果

In [5]:

# YOLO產出的預設位置:'/content/drive/MyDrive/yolo_file/darknet-master/result_real_private.txt'
with open ('/content/drive/MyDrive/_HiAdam_model/YOLOv4/result_real_private.txt', 'r') as f:
    text = f.read()
# 根據開頭 Enter Image Path:  分割每筆資料
text_sp = text.split("Enter Image Path:  ")

In [6]:
# 找檔名
raw_dict = {'nm':[],'cfd':[],'place':[]}

pattern1 = re.compile(r'data\/(.+.jpg)')
pattern2 = re.compile(r'SINOSTEEL: (\d*)%')
pattern3 = re.compile(r'left_x:\s*(-*\d+)\s*top_y:\s*(-*\d+)\s*width:\s*(-*\d+)\s*height:\s*(-*\d+)\)')
for i in text_sp:
    result1 = pattern1.findall(i)
    result2 = pattern2.findall(i)
    result3 = pattern3.findall(i)
    raw_dict['nm'].append(result1)
    raw_dict['cfd'].append(result2)
    raw_dict['place'].append(result3)
    
df = pd.DataFrame(raw_dict)
df['len'] = df['cfd'].str.len()

df[df['len']==0]

,nm,cfd,place,len
0,[],[],[],0
5971,[private_data_v2/nxWozb3DZq7DtSW6cFpzQQSpD_6I2...,[],[],0


In [7]:
# yolo無結果
df2 = df.loc[(df['len']==0) & (df['nm'].str.len()!=0),'nm']
for i in df2:
  print(i[0])

private_data_v2/nxWozb3DZq7DtSW6cFpzQQSpD_6I2.jpg


In [8]:
df = df.loc[df['len']>0,]
df['max_index'] = df.apply(lambda x:x['cfd'].index(max(x['cfd'])),axis=1) 

In [ ]:
# 方法 :　取最大

# 資料整理
df['name'] = df.apply(lambda x:x['nm'][0][16:-4] if len(x['place'])>0 else np.NaN,axis=1)

# 'xmin'
df['xmin_0'] = df.apply(lambda x:x['place'][x['max_index']][0] if len(x['place'])>0 else np.NaN,axis=1)
# 'ymin'
df['ymin_0'] = df.apply(lambda x:x['place'][x['max_index']][1] if len(x['place'])>0 else np.NaN,axis=1)

# 'w'
df['w_0'] = df.apply(lambda x:x['place'][x['max_index']][2] if len(x['place'])>0 else np.NaN,axis=1)
# 'h'
df['h_0'] = df.apply(lambda x:x['place'][x['max_index']][3] if len(x['place'])>0 else np.NaN,axis=1)
# 'cfd'
df['cfd_0'] = df.apply(lambda x:x['cfd'][x['max_index']]if len(x['cfd'])>0 else np.NaN,axis=1)
df

### 識別模型開始預測

In [ ]:
#  x (v1 -8 +16)  

ocr3 = PaddleOCR(rec_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/b/inference/',rec_char_dict_path='/content/drive/MyDrive/_HiAdam_configs/PaddleOCR/steel_en_dict.txt',rec_char_type='ch',cls_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/cls/inference',use_angle_cls=True)  # need to run only once to download and load model into memory
ocr1 = PaddleOCR(rec_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/a/inference/',rec_char_dict_path='/content/drive/MyDrive/_HiAdam_configs/PaddleOCR/steel_en_dict.txt',rec_char_type='ch',cls_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/cls/inference',use_angle_cls=True)  # need to run only once to download and load model into memory
ocr2 = PaddleOCR(rec_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/c/inference/',rec_char_dict_path='/content/drive/MyDrive/_HiAdam_configs/PaddleOCR/steel_en_dict.txt',rec_char_type='ch',cls_model_dir='/content/drive/MyDrive/_HiAdam_model/PaddleOCR/cls/inference',use_angle_cls=True)
# path = "/content/drive/MyDrive/t_brain/testing_cut" #資料夾目錄
# files= os.listdir(path)

nm = []
cls = []
cls_cfd =[]

lb1 = []
cfd1 = []
lb1_inv = []
cfd1_inv =[]

lb2 = []
cfd2 = []
lb2_inv = []
cfd2_inv =[]

lb3 = []
cfd3 = []
lb3_inv = []
cfd3_inv =[]


for _,pic in df.iterrows():
# 讀取圖檔
  print(f"{pic['name']}")
  img = cv2.imread(f"/content/drive/MyDrive/t_brain/private/data/private_data_v2/{pic['name']}.jpg")

# 裁切區域的 x 與 y 座標（左上角） : 改取min可能更好
  x = math.ceil(int(pic['xmin_0'])-8)
  if x < 0:
    x = 0
  y = math.ceil(int(pic['ymin_0']))
# 裁切區域的長度與寬度
  w = math.ceil(int(pic['w_0'])+16)
  h = math.ceil(int(pic['h_0']))

# 裁切圖片
  crop_img = img[y:y+h, x:x+w]
  # cv2.imwrite(f"/content/drive/MyDrive/t_brain/testing_cut/cut_{pic['name']}.jpg",crop_img)
  inverse_img = cv2.flip(crop_img, -1) 

  nm.append(pic['name'])

  result_cls = ocr1.ocr(crop_img, det=False, rec=False, cls=True)
  for line in result_cls:
      print(line)
  cls.append(result_cls[0][0])
  cls_cfd.append(result_cls[0][1])

  result1 = ocr1.ocr(crop_img, det=False, cls=False)
  result1_i = ocr1.ocr(inverse_img, det=False, cls=False)
  for line in result1:
      print(line)
  lb1.append(result1[0][0])
  cfd1.append(result1[0][1])
  lb1_inv.append(result1_i[0][0])
  cfd1_inv.append(result1_i[0][1])

  result2 = ocr2.ocr(crop_img, det=False, cls=False)
  result2_i = ocr2.ocr(inverse_img, det=False, cls=False)
  lb2.append(result2[0][0])
  cfd2.append(result2[0][1])
  lb2_inv.append(result2_i[0][0])
  cfd2_inv.append(result2_i[0][1])

  result3 = ocr3.ocr(crop_img, det=False, cls=False)
  result3_i = ocr3.ocr(inverse_img, det=False, cls=False)
  lb3.append(result3[0][0])
  cfd3.append(result3[0][1])
  lb3_inv.append(result3_i[0][0])
  cfd3_inv.append(result3_i[0][1])

In [ ]:
def get_best(df):
  if (df['lb_mix1'] == df['lb_mix2']) | (df['lb_mix1'] == df['lb_mix3']):
    res = df['lb_mix1']
  elif df['lb_mix2'] == df['lb_mix3']:
    res = df['lb_mix2']
  elif (df['cfd_mix1'] >= df['cfd_mix2']) & (df['cfd_mix1'] >= df['cfd_mix3']):
    res = df['lb_mix1']
  elif df['cfd_mix2'] >= df['cfd_mix3']:
    res = df['lb_mix2']
  else:
    res = df['lb_mix3']
  return res

In [ ]:
common_result = pd.DataFrame({'nm':nm,'cls':cls,'cls_cfd':cls_cfd,
                               'lb1':lb1,'cfd1':cfd1,'lb1_inv':lb1_inv,'cfd1_inv':cfd1_inv,
                               'lb2':lb2,'cfd2':cfd2,'lb2_inv':lb2_inv,'cfd2_inv':cfd2_inv,
                               'lb3':lb3,'cfd3':cfd3,'lb3_inv':lb3_inv,'cfd3_inv':cfd3_inv
                               })
common_result['cfd1'] = np.where(common_result['cls']==0,common_result['cfd1']+0.1,common_result['cfd1'])
common_result['cfd1_inv'] = np.where(common_result['cls']==180,common_result['cfd1_inv']+0.1,common_result['cfd1_inv'])
common_result['lb_mix1'] = np.where(common_result['cfd1']>common_result['cfd1_inv'],common_result['lb1'],common_result['lb1_inv'])
common_result['cfd_mix1'] = np.where(common_result['cfd1']>common_result['cfd1_inv'],common_result['cfd1'],common_result['cfd1_inv'])
                                    
common_result['cfd2'] = np.where(common_result['cls']==0,common_result['cfd2']+0.1,common_result['cfd2'])
common_result['cfd2_inv'] = np.where(common_result['cls']==180,common_result['cfd2_inv']+0.1,common_result['cfd2_inv'])
common_result['lb_mix2'] = np.where(common_result['cfd2']>common_result['cfd2_inv'],common_result['lb2'],common_result['lb2_inv'])
common_result['cfd_mix2'] = np.where(common_result['cfd2']>common_result['cfd2_inv'],common_result['cfd2'],common_result['cfd2_inv'])

common_result['cfd3'] = np.where(common_result['cls']==0,common_result['cfd3']+0.1,common_result['cfd3'])
common_result['cfd3_inv'] = np.where(common_result['cls']==180,common_result['cfd3_inv']+0.1,common_result['cfd3_inv'])
common_result['lb_mix3'] = np.where(common_result['cfd3']>common_result['cfd3_inv'],common_result['lb3'],common_result['lb3_inv'])
common_result['cfd_mix3'] = np.where(common_result['cfd3']>common_result['cfd3_inv'],common_result['cfd3'],common_result['cfd3_inv'])

common_result['lb_mix'] = common_result.apply(lambda x:get_best(x),axis=1)

common_result

### 需用平均給偵測範圍之照片

In [ ]:
pp_det = []

# 長度小於5
for _,i in common_result.iterrows():
  if len(i['lb_mix']) < 5:
    print(i['nm'])
    print(i['lb_mix'])
    pp_det.append(i['nm'])

for i in df2:
  print(i[0][16:-4])
  pp_det.append(i[0][16:-4])

In [ ]:
nm5 = []
lb5= []
cfd5 = []
lb5_inv = []
cfd5_inv =[]
cls5=[]
cls5_cfd =[]

for i in pp_det:
# 讀取圖檔
  img = cv2.imread(f"/content/drive/MyDrive/t_brain/private/data/private_data_v2/{i}.jpg")

  inverse_img = cv2.flip(img, -1) 
  result3 = ocr3.ocr(img, det=False, rec=False, cls=True)
  
  # det不到用平均位置放寬
  x = int(df['xmin_0'].astype('int').mean(axis=0)-60)
  y = int(df['ymin_0'].astype('int').mean(axis=0)-10)
  w = int(df['w_0'].astype('int').mean(axis=0)+120)
  h = int(df['h_0'].astype('int').mean(axis=0)+20)
  # 裁切圖片
  crop_img = img[y:y+h, x:x+w]
  c_inverse_img = cv2.flip(img, -1) 

  result4 = ocr2.ocr(crop_img, det=False,cls=False)
  result5 = ocr2.ocr(c_inverse_img, det=False,cls=False)

  cls5.append(result3[0][0])
  cls5_cfd.append(result3[0][1])

  lb5.append(result4[0][0])
  cfd5.append(result4[0][1])
  lb5_inv.append(result5[0][0])
  cfd5_inv.append(result5[0][1])
  nm5.append(i)

In [ ]:
common_result3 = pd.DataFrame({'nm':nm5,'lb':lb5,'cfd':cfd5,'lb_inv':lb5_inv,'cfd_inv':cfd5_inv,'cls':cls5,'cls_cfd':cls5_cfd})
common_result3['cfd'] = np.where(common_result3['cls']==0,common_result3['cfd']+0.1,common_result3['cfd'])
common_result3['cfd_inv'] = np.where(common_result3['cls']==180,common_result3['cfd_inv']+0.1,common_result3['cfd_inv'])
common_result3['lb_mix'] = np.where(common_result3['cfd']>common_result3['cfd_inv'],common_result3['lb'],common_result3['lb_inv'])
common_result3

### 最後結果整理

In [ ]:
template = pd.read_csv('/content/drive/MyDrive/t_brain/submission_template_private.csv')

result = pd.merge(template['id'],common_result,how='left',left_on='id',right_on='nm')
result = pd.merge(result,common_result3[['nm','lb_mix']],how='left',left_on='id',right_on='nm')
result['text'] = np.where(result['lb_mix_y'].isnull(),result['lb_mix_x'],result['lb_mix_y'])
# result[result['text'].isnull()]
result

In [ ]:
# 匯出
result[['id','text']].to_csv('/content/drive/MyDrive/sub_private_mix1.csv',index=False)